## Import necessary libraries 

In [443]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Sampler
from torchvision import transforms
from tqdm import tqdm  # Import tqdm for progress visualization
from models.cnn_models import SimpleCNN

## Define filepaths as constant

In [444]:
# Define file paths as constants
CSV_FILE_PATH = r'C:\Users\Sandhra George\avalanche\data\dataset.csv'
ROOT_DIR_PATH = r'C:\Users\Sandhra George\avalanche'

csv_file = r'C:\Users\Sandhra George\avalanche\data\dataset.csv'  # Path to the CSV file
root_dir = r'C:\Users\Sandhra George\avalanche\caxton_dataset\print0'  # Path to the image directory

## Load data into DataFrame

In [445]:
import pandas as pd

# Load data into a DataFrame for easier processing
data = pd.read_csv(CSV_FILE_PATH)

# Limit dataset to the first 3084 images (excluding header)
data_limited = data.iloc[0:3085].reset_index(drop=True)

# Filter the dataset to only include images containing "print0"
data_filtered = data_limited[data_limited.iloc[:, 0].str.contains('print0', na=False)]

# Update the first column to contain only the image filenames
data_filtered.iloc[:, 0] = data_filtered.iloc[:, 0].str.replace(r'.*?/(image-\d+\.jpg)', r'\1', regex=True)

# Display the updated DataFrame
print(data_filtered.head())

       img_path               timestamp  flow_rate  feed_rate  z_offset  \
0   image-6.jpg  2020-10-08T13:12:50-34        100        100       0.0   
1   image-7.jpg  2020-10-08T13:12:50-80        100        100       0.0   
2   image-8.jpg  2020-10-08T13:12:51-27        100        100       0.0   
3   image-9.jpg  2020-10-08T13:12:51-74        100        100       0.0   
4  image-10.jpg  2020-10-08T13:12:52-20        100        100       0.0   

   target_hotend  hotend    bed  nozzle_tip_x  nozzle_tip_y  img_num  \
0          205.0  204.13  65.74           531           554        5   
1          205.0  204.13  65.74           531           554        6   
2          205.0  204.24  65.84           531           554        7   
3          205.0  204.24  65.84           531           554        8   
4          205.0  204.24  65.84           531           554        9   

   print_id  flow_rate_class  feed_rate_class  z_offset_class  hotend_class  \
0         0                1         

## Split the dataset into separate DataFrames for each class

In [446]:
class_datasets = {}
for class_id in data_filtered['hotend_class'].unique():
    # Print the unique class identifier
    print(f"Processing class: {class_id}")
    
    # Create a shuffled subset for the current class
    class_datasets[class_id] = data_filtered[data_filtered['hotend_class'] == class_id].sample(frac=1, random_state=42)
    
# Print counts of each class dataset
for class_id, df in class_datasets.items():
    print(f'Class {class_id} dataset size: {len(df)}')

# Initialize variables to track the minimum class size and the corresponding class
min_class_size = float('inf')  # Start with infinity as a comparison baseline
min_class_id = None  # Variable to hold the class ID with the minimum size

# Iterate over class datasets to find the minimum class size and its corresponding class ID
for class_id, df in class_datasets.items():
    class_size = len(df)
    if class_size < min_class_size:
        min_class_size = class_size
        min_class_id = class_id

# Print the minimum class size and the corresponding class ID
print(f'Minimum class size: {min_class_size} (Class: {min_class_id})')

Processing class: 1
Processing class: 0
Processing class: 2
Class 1 dataset size: 1279
Class 0 dataset size: 710
Class 2 dataset size: 721
Minimum class size: 710 (Class: 0)


## Create a balanced dataset

In [447]:
# Create balanced datasets by taking the minimum number of images from each class
balanced_data = []
for class_id, class_data in class_datasets.items():
    # Sample from each class
    balanced_data.append(class_data.sample(n=min_class_size, random_state=42))

# Print the number of images from each class in the balanced dataset
for i, class_data in enumerate(balanced_data):
    print(f'Class {list(class_datasets.keys())[i]} dataset size: {len(class_data)}')

# Combine the balanced data into a single DataFrame
balanced_data = pd.concat(balanced_data).reset_index(drop=True)

# Shuffle the balanced dataset
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

Class 1 dataset size: 710
Class 0 dataset size: 710
Class 2 dataset size: 710


## Create training, validation, and testing datasets

In [448]:
# Total number of images in the balanced dataset
total_images = len(balanced_data)

# Print the total number of images
print(f'Total images: {total_images}')

# Get the minimum number of samples available in any class
min_class_counts = balanced_data['hotend_class'].value_counts().min()

# Define how many samples you want from each class in each dataset
train_samples_per_class = int(0.8 * min_class_counts)
val_samples_per_class = int(0.1 * min_class_counts)
test_samples_per_class = min_class_counts - train_samples_per_class - val_samples_per_class

# Debug: Print the sizes for each class in each dataset
print(f'Minimum samples per class: {min_class_counts}')
print(f'Samples per class - Train: {train_samples_per_class}, Validation: {val_samples_per_class}, Test: {test_samples_per_class}')

# Initialize empty DataFrames for each dataset
train_data = pd.DataFrame()
val_data = pd.DataFrame()
test_data = pd.DataFrame()

# Sample data for each class
for class_id in balanced_data['hotend_class'].unique():
    class_data = balanced_data[balanced_data['hotend_class'] == class_id]
    
    # Shuffle the class data
    class_data = class_data.sample(frac=1).reset_index(drop=True)  # Shuffle the class data

    # Split the data into train, val, and test
    train_subset = class_data.iloc[:train_samples_per_class]
    val_subset = class_data.iloc[train_samples_per_class:train_samples_per_class + val_samples_per_class]
    test_subset = class_data.iloc[train_samples_per_class + val_samples_per_class:train_samples_per_class + val_samples_per_class + test_samples_per_class]

    # Append to respective datasets
    train_data = pd.concat([train_data, train_subset], ignore_index=True)
    val_data = pd.concat([val_data, val_subset], ignore_index=True)
    test_data = pd.concat([test_data, test_subset], ignore_index=True)

# Debug: Print the sizes of the datasets after the split
print(f'Training set size: {len(train_data)}')
print(f'Validation set size: {len(val_data)}')
print(f'Testing set size: {len(test_data)}')

# Function to print class counts for a given dataset
def print_class_counts(dataset, dataset_name):
    class_counts = dataset['hotend_class'].value_counts()
    print(f"\nClass indices count in {dataset_name}:")
    for class_id, count in class_counts.items():
        print(f'Class {class_id}: {count} images')

# Print class counts for each dataset
print_class_counts(train_data, "training data")
print_class_counts(val_data, "validation data")
print_class_counts(test_data, "testing data")

# Print the first five rows of each dataset
print("\nFirst five rows of training data:")
print(train_data.head())

print("\nFirst five rows of validation data:")
print(val_data.head())

print("\nFirst five rows of testing data:")
print(test_data.head())

Total images: 2130
Minimum samples per class: 710
Samples per class - Train: 568, Validation: 71, Test: 71
Training set size: 1704
Validation set size: 213
Testing set size: 213

Class indices count in training data:
Class 1: 568 images
Class 2: 568 images
Class 0: 568 images

Class indices count in validation data:
Class 1: 71 images
Class 2: 71 images
Class 0: 71 images

Class indices count in testing data:
Class 1: 71 images
Class 2: 71 images
Class 0: 71 images

First five rows of training data:
         img_path               timestamp  flow_rate  feed_rate  z_offset  \
0   image-759.jpg  2020-10-08T13:18:41-49        109        190     -0.02   
1  image-1161.jpg  2020-10-08T13:21:48-84        157        140     -0.05   
2   image-686.jpg  2020-10-08T13:18:07-24        109        190     -0.02   
3  image-1182.jpg  2020-10-08T13:21:58-60        157        140     -0.05   
4  image-2141.jpg  2020-10-08T13:29:25-25        149        200      0.14   

   target_hotend  hotend    bed 

## Initialise model, loss function, and optimiser

In [449]:
num_classes = 3  # Number of hot end rate classes
model = SimpleCNN(num_classes=num_classes)  # Assuming SimpleCNN is defined in cnn_models
criterion = nn.CrossEntropyLoss()  # Cross Entropy Loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

## Creating a Balanced batch sampler class

In [450]:
import numpy as np
import pandas as pd
from torch.utils.data import Sampler

class BalancedBatchSampler(Sampler):
    def __init__(self, data_source, batch_size=15):
        self.data_source = data_source
        self.batch_size = batch_size

        # Ensure the batch size is evenly divisible by the number of classes
        self.num_classes = len(data_source['hotend_class'].unique())
        if self.batch_size % self.num_classes != 0:
            raise ValueError("Batch size must be divisible by the number of classes.")
        
        self.samples_per_class = self.batch_size // self.num_classes
        
        # Group indices by class
        self.class_indices = {
            class_id: np.array(data_source.index[data_source['hotend_class'] == class_id])
            for class_id in data_source['hotend_class'].unique()
        }
        
        # Shuffle class indices
        for class_id in self.class_indices:
            np.random.shuffle(self.class_indices[class_id])

        # Debug: Print class indices and their counts
        print(f'Class indices: {self.class_indices}')

        # Initialize counters for balanced and imbalanced batches
        self.balanced_batches_count = 0
        self.imbalanced_batches_count = 0

    def __len__(self):
        # Calculate the total number of batches
        min_class_samples = min(len(indices) for indices in self.class_indices.values())
        return min_class_samples // self.samples_per_class

    def __iter__(self):
        while True:
            batch = []
            class_count = {class_id: 0 for class_id in self.class_indices.keys()}  # Initialize class count
            
            all_classes_filled = True  # Flag to check if all classes have enough samples
            
            for class_id, indices in self.class_indices.items():
                if len(indices) < self.samples_per_class:
                    all_classes_filled = False  # Not enough samples for this class
                    break  # Exit if any class doesn't have enough samples
                
                # Take samples for the batch from each class
                batch.extend(indices[:self.samples_per_class])
                class_count[class_id] += self.samples_per_class  # Count the samples added to the batch
                # Remove these samples from the class indices
                self.class_indices[class_id] = indices[self.samples_per_class:]

            if not all_classes_filled:  # Exit if not all classes have enough samples
                break
            
            # Shuffle within the batch
            np.random.shuffle(batch)

            # Print the number of images from each class in the current batch
            print(f'Batch class counts: {class_count}')

            # Check if the current batch is balanced or imbalanced
            if all(count == self.samples_per_class for count in class_count.values()):
                self.balanced_batches_count += 1
            else:
                self.imbalanced_batches_count += 1

            yield batch  # Yield the current batch

    def print_batch_counts(self):
        # Print the counts of balanced and imbalanced batches
        print(f'Total balanced batches: {self.balanced_batches_count}')
        print(f'Total imbalanced batches: {self.imbalanced_batches_count}')
        
# Usage example
# Assuming balanced_data is already defined as per your previous code
sampler = BalancedBatchSampler(balanced_data, batch_size=15)

# Example of getting all batches
for batch_indices in sampler:
    print(f'Batch indices: {batch_indices}')  # This will show all indices in the batch
    # Here you can use these indices to get the corresponding images from balanced_data
    images = balanced_data.iloc[batch_indices]  # Access the images using the batch indices

# After iterating through all batches, print the totals
sampler.print_batch_counts()  # Call this method to print the totals

Class indices: {1: array([ 689,   87,  474, 1525, 1270, 1960,  527, 1250,  470, 1865,   88,
       1751,  755,  965,  183,  639,  525,   76,  156,  735,  246, 2038,
       2095,  582, 1340, 1208,   72, 2122, 1726,  351, 1176, 1638, 1248,
        707, 2007,  634,  749,  810,  865,  752, 1303,  459, 1381, 1474,
        566, 1701, 1508,  505,  469,  166, 1143, 1329,  778, 1692,  859,
       1549, 1603,  799, 1616,  264,  838, 1055,  608, 1771,  411,  925,
        844, 1195, 1036,  645, 1844, 1509, 1427, 1102, 1114,  740, 1184,
       1121, 1354,   46,  609, 1280,  530, 1906,  968, 1733,  325,  472,
       1720, 2097,  781, 1173, 1627, 2055,  734, 2003,  667, 1596,  756,
       1452,  521, 1228, 1199,  908,  912,  661, 1504, 2067, 1166,  638,
       1119,  448, 2012, 1503, 1016, 1203,  477,   44,  467, 1523,  414,
       1810,  574, 1214, 1170, 1062, 1858, 2014,  948, 1736, 1310, 1620,
        433, 1194, 1871,  535,  742, 1686, 1145,  408, 1434, 2083,  513,
        111, 1602,  618, 1575,  

## Training, Validation and Testing batches

In [451]:
from torch.utils.data import Dataset
import pandas as pd
from PIL import Image
from tqdm import tqdm
import os
from torchvision import transforms
import torch

class CustomDataset(Dataset):
    def __init__(self, csv_file=None, root_dir=None, transform=None, data_frame=None):
        if data_frame is not None:
            self.data = data_frame
        elif csv_file is not None:
            self.data = pd.read_csv(csv_file, header=0, dtype=str)
        else:
            raise ValueError("Either csv_file or data_frame must be provided.")

        self.root_dir = root_dir
        self.transform = transform or self.default_transform()
        self.valid_indices = self.get_valid_indices()

    def default_transform(self):
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
        ])

    def get_valid_indices(self):
        valid_indices = []
        for idx in tqdm(range(len(self.data)), desc="Validating images"):
            img_name = self.data.iloc[idx, 0].strip()
            img_name = img_name.split('/')[-1]
    
            if img_name.startswith("image-"):
                try:
                    image_number = int(img_name.split('-')[1].split('.')[0])
                    if image_number <= 3084:
                        full_img_path = os.path.join(self.root_dir, img_name)
                        if os.path.exists(full_img_path):
                            valid_indices.append(idx)
                        else:
                            print(f"Image does not exist: {full_img_path}")
                except ValueError:
                    print(f"Invalid filename format for {img_name}. Skipping...")
        
        print(f"Total valid indices found: {len(valid_indices)}")  # Debugging output
        return valid_indices


    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        if isinstance(idx, list):
            # Debugging: Print length of valid indices
            print(f"Valid indices count: {len(self.valid_indices)}")
    
            items = [self._load_sample(i) for i in idx]
            items = [item for item in items if item is not None]  # Filter out None entries
    
            if not items:
                raise RuntimeError("No valid items found in the batch.")
    
            # Unzip items and stack images
            images, labels = zip(*items)
            return torch.stack(images), torch.tensor(labels)
    
        else:
            return self._load_sample(idx)


    def _load_sample(self, idx):
        # Get the actual index from valid indices
        actual_idx = self.valid_indices[idx]
        img_name = self.data.iloc[actual_idx, 0].strip()
        full_img_path = os.path.join(self.root_dir, img_name)
    
        try:
            image = Image.open(full_img_path).convert('RGB')  # Ensure image is RGB
            label_str = self.data.iloc[actual_idx, 15]  # Assuming label is in the second column
            
            # Attempt to convert label to integer; handle exceptions
            try:
                label = int(label_str)  # Try converting to int
            except ValueError:
                print(f"Warning: Non-integer label found for image {img_name}: {label_str}")
                print()
                return None  # Skip this sample if label conversion fails
    
            image = self.transform(image)  # Apply transformation
    
            return image, label
        except Exception as e:
            print(f"Error loading image {full_img_path}: {e}")
            return None  # Handle error gracefully

In [455]:
# Paths and configuration
root_dir = r'C:\Users\Sandhra George\avalanche\caxton_dataset\print0'  # Path to the image directory
batch_size = 15

# Wrap each dataset split in CustomDataset
train_dataset = CustomDataset(data_frame=train_data, root_dir=root_dir)
val_dataset = CustomDataset(data_frame=val_data, root_dir=root_dir)

# Initialize the BalancedBatchSampler for the train_dataset
train_sampler = BalancedBatchSampler(train_data, batch_size=batch_size)
val_sampler = BalancedBatchSampler(val_data, batch_size=batch_size)

# Create DataLoaders with the sampler for balanced batches in training
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, sampler=val_sampler)

print_class_counts(train_data, "train_data")
print_class_counts(val_data, "val_data")

Validating images: 100%|██████████| 1704/1704 [00:00<00:00, 11681.90it/s]


Total valid indices found: 1704


Validating images: 100%|██████████| 213/213 [00:00<00:00, 2689.15it/s]

Total valid indices found: 213
Class indices: {1: array([326, 216, 413, 223,  84, 380, 307, 529, 451, 285, 395, 169, 123,
       254, 194, 171, 255,  70, 432, 233, 296,  15,  37, 510, 263, 434,
       518, 114, 135, 429, 562, 383, 234, 542, 232, 253, 427,  78, 426,
       249, 215, 260, 192, 308, 377, 279, 248, 122, 402, 153, 241, 417,
       406,  99, 302, 336,  38,  44,   6, 356, 129, 257, 408, 546, 116,
         0,  22, 364, 410, 538,   1, 433, 425, 217,  17,  31, 450, 218,
       409,  91, 178, 455, 289, 547, 187, 349, 448, 134, 500, 284, 544,
       559, 220, 495, 405, 397, 151, 483, 554, 464,  95, 381, 531, 334,
       536,  39,  74, 262,  75, 204, 461, 117, 468, 460, 306, 355, 292,
       492,   8, 360, 286, 166,  41, 565, 419, 553, 488,  82,   9, 431,
       225,  71, 290,  25, 347, 201, 199, 154, 342, 127, 378, 520, 511,
       491,  87, 132, 373, 331, 351, 301, 164, 525, 108, 566, 452,  72,
       258, 185, 144, 131,  58, 197, 392, 385, 519, 459, 319, 100,  43,
       226,  6

In [456]:
def create_dataloader(csv_file, root_dir, batch_size, transform):
    # Create an instance of the CustomDataset
    dataset = CustomDataset(csv_file=csv_file, root_dir=root_dir, transform=transform)

    # Create the DataLoader
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Print the total number of valid images and batches
    print(f'Total valid images: {len(dataset)}')
    print(f'Dataloader size (number of batches 1): {len(dataloader)}')

    return dataloader

In [467]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Iterate through one batch for demonstration
try:
    for images, labels in train_dataloader:
        print(f'Batch images shape: {images.shape}, Batch labels: {labels}')
        break  # Remove this break to iterate through the whole dataloader
except Exception as e:
    print(f"Error during DataLoader iteration: {e}")


Batch images shape: torch.Size([15, 3, 224, 224]), Batch labels: tensor([1, 2, 2, 2, 2, 0, 2, 2, 1, 1, 2, 1, 2, 0, 0])


In [470]:
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Iterate through one batch for demonstration
try:
    for images, labels in val_dataloader:
        print(f'Batch images shape: {images.shape}, Batch labels: {labels}')
        break  # Remove this break to iterate through the whole dataloader
except Exception as e:
    print(f"Error during DataLoader iteration: {e}")

Batch images shape: torch.Size([15, 3, 224, 224]), Batch labels: tensor([0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 0, 0, 0, 2, 1])


## Debug: Check if DataLoader has zero length

In [474]:
print(f'Train loader size: {len(train_dataloader)}')
print(f'Validation loader size: {len(val_dataloader)}')

Train loader size: 114
Validation loader size: 15


## Beginning of training loop with validation

In [ ]:
num_epochs = 5  # Number of epochs for training

In [ ]:
for epoch in range(num_epochs):
    print(f'Training Epoch {epoch + 1}/{num_epochs}:')

    # Training phase
    model.train()  # Set the model to training mode
    epoch_loss = 0  # Initialize epoch loss
    for images, labels in tqdm(train_dataloader, desc='Training', leave=True, dynamic_ncols=True):
        optimizer.zero_grad()  # Reset gradients
        outputs = model(images)  # Forward pass
        loss = criterion(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        epoch_loss += loss.item()  # Accumulate loss

    avg_epoch_loss = epoch_loss / len(train_dataloader)
    print(f'Average Training Loss: {avg_epoch_loss:.4f}')

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for images, labels in tqdm(val_dataloader, desc='Validation', leave=True, dynamic_ncols=True):
            outputs = model(images)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            val_loss += loss.item()  # Accumulate validation loss

            _, predicted = torch.max(outputs, 1)  # Get the class with the highest score
            correct_predictions += (predicted == labels).sum().item()  # Count correct predictions
            total_predictions += labels.size(0)  # Update total predictions

    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = correct_predictions / total_predictions
    print(f'Average Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.4f}')


Training Epoch 1/5:


Training:  11%|█▏        | 13/114 [00:26<03:18,  1.97s/it]